In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.columns = [each.lower() for each in df.columns]
df['abstract']  = df['title'].apply(lambda x: x + ". ") + df['abstract']

In [4]:
test_size = 0.15

indices = list(df.index)
np.random.shuffle(indices)

test_indices  = indices[:int(test_size*len(indices))]
train_indices = indices[int(test_size*len(indices)):]

In [5]:
train_texts = df.loc[train_indices, 'abstract']
test_texts  = df.loc[test_indices, 'abstract']

In [6]:
# Stemming
from nltk.stem import PorterStemmer

ps = PorterStemmer()


In [7]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
train_texts_processed = []
for i,each_line in enumerate(train_texts):
  #print (each_line)
  line = []
  for each_word in text_to_word_sequence(each_line.lower()):
    line.append(ps.stem(each_word))
  new_line = " ".join(line)
  train_texts_processed.append(new_line)


test_texts_processed = []
for i,each_line in enumerate(test_texts):
  #print (each_line)
  line = []
  for each_word in text_to_word_sequence(each_line.lower()):
    line.append(ps.stem(each_word))
  new_line = " ".join(line)
  test_texts_processed.append(new_line)


In [8]:
#6 columns of onehot vectors, each one for - computer science,physics,mathematics,statistics,quantitative biology,quantitative finance
n_labels = 6
labels = np.zeros((df.shape[0], n_labels))
for i,each_label in enumerate(df.columns[3:]):
  print (each_label,i)
  labels[:,i] = df.loc[:,each_label].values

computer science 0
physics 1
mathematics 2
statistics 3
quantitative biology 4
quantitative finance 5


In [9]:
#Train-Test labels
train_labels = labels[train_indices,:]
test_labels  = labels[test_indices,:]

# Vocab selection for last two labels

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from collections import Counter

positive_counts = Counter()
negative_counts = Counter()
total_counts    = Counter() 

In [11]:
for i,each_text in enumerate(train_texts_processed):
  if labels[i,-2] == 1:#Positive
    for each_word in text_to_word_sequence(each_text.lower()):
      positive_counts[each_word] += 1
      total_counts[each_word] += 1
  if labels[i,-2] == 0:#Positive
    for each_word in text_to_word_sequence(each_text.lower()):
      negative_counts[each_word] += 1
      total_counts[each_word] += 1

In [12]:
pos_neg_ratios = Counter()

for term, cnt in list(total_counts.most_common()):
  if cnt>50:
    pos_neg_ratio = positive_counts[term] / float( negative_counts[term] + 1 )
    pos_neg_ratios[term] = pos_neg_ratio

In [13]:
for word, ratio in pos_neg_ratios.most_common():
  if ratio > 1:
    pos_neg_ratio[word] = np.log(ratio)
  else:
    pos_neg_ratios[word] = -np.log((1 / (ratio+0.01)))

In [14]:
vocab = [k for k,v in pos_neg_ratios.items()]
len(vocab)

3442

In [15]:
vocab[:10]

['the', 'of', 'and', 'a', 'in', 'to', 'we', 'for', 'is', 'that']

In [16]:
import nltk
from nltk.corpus import stopwords
StopWords = stopwords.words('english')

In [17]:
v2 = [v for v in vocab if v not in StopWords]
len(v2)

3346

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(train_texts_processed)
train_vec = vectorizer.transform(train_texts_processed)
test_vec  = vectorizer.transform(test_texts_processed)

In [19]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression

In [20]:
df_test = pd.read_csv('test.csv')
df_test['ABSTRACT']  = df_test['TITLE'].apply(lambda x: x + ". ") + df_test['ABSTRACT']
texts = df_test.loc[:, 'ABSTRACT']

texts_processed = []
for i,each_line in enumerate(texts):
  #print (each_line)
  line = []
  for each_word in text_to_word_sequence(each_line.lower()):
    line.append(ps.stem(each_word))
  new_line = " ".join(line)
  texts_processed.append(new_line)

vec = vectorizer.transform(texts_processed)

In [21]:
from catboost import CatBoostClassifier

In [22]:
f1_arr = []
pred_arr=[]
for i in range(6):
  weight = np.unique(train_labels[:,i], return_counts=True)[1][0]/np.unique(train_labels[:,i], return_counts=True)[1][1]
  #if weight > 10:
  #  weight = weight/3
  if i == 4:
    weight = weight/17
  if i == 5:
    weight = weight/27
  model = CatBoostClassifier(iterations = 1000, learning_rate=0.1, depth=8,scale_pos_weight=weight)
  model.fit(train_vec, train_labels[:,i], eval_set=(test_vec, test_labels[:,i]))
  #model  = LogisticRegression()
  #model.fit(train_vec, train_labels[:,i])


  y_pred_train = model.predict(train_vec)
  f1  = f1_score(train_labels[:,i], y_pred_train)
  f1_arr.append(f1)
  print (i, f1)
  
  pred = model.predict(vec)
  pred_arr.append(pred)

0:	learn: 0.6573417	test: 0.6575638	best: 0.6575638 (0)	total: 479ms	remaining: 7m 58s
1:	learn: 0.6251216	test: 0.6249925	best: 0.6249925 (1)	total: 871ms	remaining: 7m 14s
2:	learn: 0.6037025	test: 0.6025402	best: 0.6025402 (2)	total: 1.24s	remaining: 6m 50s
3:	learn: 0.5853207	test: 0.5840036	best: 0.5840036 (3)	total: 1.61s	remaining: 6m 40s
4:	learn: 0.5700126	test: 0.5688526	best: 0.5688526 (4)	total: 1.97s	remaining: 6m 32s
5:	learn: 0.5566445	test: 0.5550172	best: 0.5550172 (5)	total: 2.31s	remaining: 6m 22s
6:	learn: 0.5448886	test: 0.5434837	best: 0.5434837 (6)	total: 2.67s	remaining: 6m 19s
7:	learn: 0.5331632	test: 0.5313852	best: 0.5313852 (7)	total: 3.07s	remaining: 6m 20s
8:	learn: 0.5227077	test: 0.5211599	best: 0.5211599 (8)	total: 3.42s	remaining: 6m 16s
9:	learn: 0.5136191	test: 0.5115330	best: 0.5115330 (9)	total: 3.78s	remaining: 6m 14s
10:	learn: 0.5057148	test: 0.5033991	best: 0.5033991 (10)	total: 4.15s	remaining: 6m 12s
11:	learn: 0.4987895	test: 0.4964247	best

93:	learn: 0.3506235	test: 0.3633416	best: 0.3633416 (93)	total: 34.3s	remaining: 5m 30s
94:	learn: 0.3496280	test: 0.3625477	best: 0.3625477 (94)	total: 34.7s	remaining: 5m 30s
95:	learn: 0.3485533	test: 0.3619099	best: 0.3619099 (95)	total: 35.1s	remaining: 5m 30s
96:	learn: 0.3479506	test: 0.3613220	best: 0.3613220 (96)	total: 35.5s	remaining: 5m 30s
97:	learn: 0.3471797	test: 0.3606464	best: 0.3606464 (97)	total: 35.9s	remaining: 5m 30s
98:	learn: 0.3465223	test: 0.3599919	best: 0.3599919 (98)	total: 36.2s	remaining: 5m 29s
99:	learn: 0.3457185	test: 0.3593687	best: 0.3593687 (99)	total: 36.6s	remaining: 5m 29s
100:	learn: 0.3445458	test: 0.3586657	best: 0.3586657 (100)	total: 37s	remaining: 5m 28s
101:	learn: 0.3435672	test: 0.3586545	best: 0.3586545 (101)	total: 37.3s	remaining: 5m 28s
102:	learn: 0.3424905	test: 0.3581060	best: 0.3581060 (102)	total: 37.7s	remaining: 5m 28s
103:	learn: 0.3418049	test: 0.3577974	best: 0.3577974 (103)	total: 38.1s	remaining: 5m 28s
104:	learn: 0.3

184:	learn: 0.2859960	test: 0.3358312	best: 0.3358312 (184)	total: 1m 7s	remaining: 4m 57s
185:	learn: 0.2853659	test: 0.3354863	best: 0.3354863 (185)	total: 1m 7s	remaining: 4m 57s
186:	learn: 0.2849304	test: 0.3354680	best: 0.3354680 (186)	total: 1m 8s	remaining: 4m 56s
187:	learn: 0.2842409	test: 0.3352534	best: 0.3352534 (187)	total: 1m 8s	remaining: 4m 56s
188:	learn: 0.2837764	test: 0.3352503	best: 0.3352503 (188)	total: 1m 9s	remaining: 4m 56s
189:	learn: 0.2831626	test: 0.3353651	best: 0.3352503 (188)	total: 1m 9s	remaining: 4m 55s
190:	learn: 0.2827014	test: 0.3353472	best: 0.3352503 (188)	total: 1m 9s	remaining: 4m 55s
191:	learn: 0.2820488	test: 0.3350883	best: 0.3350883 (191)	total: 1m 10s	remaining: 4m 55s
192:	learn: 0.2814485	test: 0.3350514	best: 0.3350514 (192)	total: 1m 10s	remaining: 4m 54s
193:	learn: 0.2812703	test: 0.3351494	best: 0.3350514 (192)	total: 1m 10s	remaining: 4m 54s
194:	learn: 0.2806977	test: 0.3349013	best: 0.3349013 (194)	total: 1m 11s	remaining: 4m

274:	learn: 0.2478523	test: 0.3270389	best: 0.3270389 (274)	total: 1m 40s	remaining: 4m 25s
275:	learn: 0.2473709	test: 0.3268354	best: 0.3268354 (275)	total: 1m 40s	remaining: 4m 24s
276:	learn: 0.2472827	test: 0.3268531	best: 0.3268354 (275)	total: 1m 41s	remaining: 4m 24s
277:	learn: 0.2469316	test: 0.3267474	best: 0.3267474 (277)	total: 1m 41s	remaining: 4m 24s
278:	learn: 0.2468418	test: 0.3267506	best: 0.3267474 (277)	total: 1m 42s	remaining: 4m 23s
279:	learn: 0.2464039	test: 0.3265816	best: 0.3265816 (279)	total: 1m 42s	remaining: 4m 23s
280:	learn: 0.2459205	test: 0.3263238	best: 0.3263238 (280)	total: 1m 42s	remaining: 4m 22s
281:	learn: 0.2453723	test: 0.3260876	best: 0.3260876 (281)	total: 1m 43s	remaining: 4m 22s
282:	learn: 0.2450013	test: 0.3258758	best: 0.3258758 (282)	total: 1m 43s	remaining: 4m 22s
283:	learn: 0.2449241	test: 0.3258749	best: 0.3258749 (283)	total: 1m 43s	remaining: 4m 21s
284:	learn: 0.2447591	test: 0.3258680	best: 0.3258680 (284)	total: 1m 44s	remain

364:	learn: 0.2220953	test: 0.3212879	best: 0.3212879 (364)	total: 2m 13s	remaining: 3m 52s
365:	learn: 0.2220383	test: 0.3212847	best: 0.3212847 (365)	total: 2m 13s	remaining: 3m 51s
366:	learn: 0.2217350	test: 0.3212792	best: 0.3212792 (366)	total: 2m 14s	remaining: 3m 51s
367:	learn: 0.2216695	test: 0.3212448	best: 0.3212448 (367)	total: 2m 14s	remaining: 3m 51s
368:	learn: 0.2211892	test: 0.3211200	best: 0.3211200 (368)	total: 2m 14s	remaining: 3m 50s
369:	learn: 0.2208598	test: 0.3208810	best: 0.3208810 (369)	total: 2m 15s	remaining: 3m 50s
370:	learn: 0.2207542	test: 0.3208630	best: 0.3208630 (370)	total: 2m 15s	remaining: 3m 49s
371:	learn: 0.2204848	test: 0.3207057	best: 0.3207057 (371)	total: 2m 15s	remaining: 3m 49s
372:	learn: 0.2203951	test: 0.3207163	best: 0.3207057 (371)	total: 2m 16s	remaining: 3m 49s
373:	learn: 0.2199411	test: 0.3205192	best: 0.3205192 (373)	total: 2m 16s	remaining: 3m 48s
374:	learn: 0.2195984	test: 0.3202301	best: 0.3202301 (374)	total: 2m 17s	remain

...

611:	learn: 0.0016054	test: 0.0694491	best: 0.0593624 (171)	total: 2m 38s	remaining: 1m 40s
612:	learn: 0.0015981	test: 0.0694442	best: 0.0593624 (171)	total: 2m 38s	remaining: 1m 40s
613:	learn: 0.0015964	test: 0.0694696	best: 0.0593624 (171)	total: 2m 38s	remaining: 1m 39s
614:	learn: 0.0015948	test: 0.0694925	best: 0.0593624 (171)	total: 2m 38s	remaining: 1m 39s
615:	learn: 0.0015946	test: 0.0694636	best: 0.0593624 (171)	total: 2m 39s	remaining: 1m 39s
616:	learn: 0.0015946	test: 0.0694633	best: 0.0593624 (171)	total: 2m 39s	remaining: 1m 38s
617:	learn: 0.0015945	test: 0.0694470	best: 0.0593624 (171)	total: 2m 39s	remaining: 1m 38s
618:	learn: 0.0015943	test: 0.0694216	best: 0.0593624 (171)	total: 2m 39s	remaining: 1m 38s
619:	learn: 0.0015943	test: 0.0694184	best: 0.0593624 (171)	total: 2m 40s	remaining: 1m 38s
620:	learn: 0.0015942	test: 0.0694160	best: 0.0593624 (171)	total: 2m 40s	remaining: 1m 37s
621:	learn: 0.0015942	test: 0.0694153	best: 0.0593624 (171)	total: 2m 40s	remain

701:	learn: 0.0015672	test: 0.0692540	best: 0.0593624 (171)	total: 3m	remaining: 1m 16s
702:	learn: 0.0015670	test: 0.0692521	best: 0.0593624 (171)	total: 3m	remaining: 1m 16s
703:	learn: 0.0015669	test: 0.0692501	best: 0.0593624 (171)	total: 3m 1s	remaining: 1m 16s
704:	learn: 0.0015667	test: 0.0692487	best: 0.0593624 (171)	total: 3m 1s	remaining: 1m 15s
705:	learn: 0.0015666	test: 0.0692471	best: 0.0593624 (171)	total: 3m 1s	remaining: 1m 15s
706:	learn: 0.0015664	test: 0.0692462	best: 0.0593624 (171)	total: 3m 1s	remaining: 1m 15s
707:	learn: 0.0015663	test: 0.0692453	best: 0.0593624 (171)	total: 3m 2s	remaining: 1m 15s
708:	learn: 0.0015661	test: 0.0692438	best: 0.0593624 (171)	total: 3m 2s	remaining: 1m 14s
709:	learn: 0.0015660	test: 0.0692427	best: 0.0593624 (171)	total: 3m 2s	remaining: 1m 14s
710:	learn: 0.0015659	test: 0.0692422	best: 0.0593624 (171)	total: 3m 3s	remaining: 1m 14s
711:	learn: 0.0015657	test: 0.0692414	best: 0.0593624 (171)	total: 3m 3s	remaining: 1m 14s
712:	

792:	learn: 0.0015028	test: 0.0695781	best: 0.0593624 (171)	total: 3m 23s	remaining: 53.1s
793:	learn: 0.0015028	test: 0.0695775	best: 0.0593624 (171)	total: 3m 23s	remaining: 52.8s
794:	learn: 0.0015028	test: 0.0695774	best: 0.0593624 (171)	total: 3m 23s	remaining: 52.5s
795:	learn: 0.0015028	test: 0.0695740	best: 0.0593624 (171)	total: 3m 23s	remaining: 52.3s
796:	learn: 0.0015028	test: 0.0695737	best: 0.0593624 (171)	total: 3m 24s	remaining: 52s
797:	learn: 0.0015028	test: 0.0695725	best: 0.0593624 (171)	total: 3m 24s	remaining: 51.7s
798:	learn: 0.0015028	test: 0.0695719	best: 0.0593624 (171)	total: 3m 24s	remaining: 51.5s
799:	learn: 0.0015028	test: 0.0695718	best: 0.0593624 (171)	total: 3m 24s	remaining: 51.2s
800:	learn: 0.0015028	test: 0.0695706	best: 0.0593624 (171)	total: 3m 25s	remaining: 51s
801:	learn: 0.0015028	test: 0.0695706	best: 0.0593624 (171)	total: 3m 25s	remaining: 50.7s
802:	learn: 0.0015028	test: 0.0695700	best: 0.0593624 (171)	total: 3m 25s	remaining: 50.4s
803

883:	learn: 0.0015025	test: 0.0695051	best: 0.0593624 (171)	total: 3m 44s	remaining: 29.5s
884:	learn: 0.0015025	test: 0.0695050	best: 0.0593624 (171)	total: 3m 44s	remaining: 29.2s
885:	learn: 0.0015025	test: 0.0695048	best: 0.0593624 (171)	total: 3m 45s	remaining: 29s
886:	learn: 0.0015025	test: 0.0695041	best: 0.0593624 (171)	total: 3m 45s	remaining: 28.7s
887:	learn: 0.0015025	test: 0.0695040	best: 0.0593624 (171)	total: 3m 45s	remaining: 28.5s
888:	learn: 0.0015025	test: 0.0695036	best: 0.0593624 (171)	total: 3m 45s	remaining: 28.2s
889:	learn: 0.0015025	test: 0.0695032	best: 0.0593624 (171)	total: 3m 46s	remaining: 27.9s
890:	learn: 0.0015025	test: 0.0695010	best: 0.0593624 (171)	total: 3m 46s	remaining: 27.7s
891:	learn: 0.0015025	test: 0.0695006	best: 0.0593624 (171)	total: 3m 46s	remaining: 27.4s
892:	learn: 0.0015025	test: 0.0694999	best: 0.0593624 (171)	total: 3m 46s	remaining: 27.2s
893:	learn: 0.0015025	test: 0.0694995	best: 0.0593624 (171)	total: 3m 46s	remaining: 26.9s
8

974:	learn: 0.0014335	test: 0.0697403	best: 0.0593624 (171)	total: 4m 5s	remaining: 6.31s
975:	learn: 0.0014335	test: 0.0697401	best: 0.0593624 (171)	total: 4m 6s	remaining: 6.05s
976:	learn: 0.0014335	test: 0.0697401	best: 0.0593624 (171)	total: 4m 6s	remaining: 5.8s
977:	learn: 0.0014335	test: 0.0697398	best: 0.0593624 (171)	total: 4m 6s	remaining: 5.55s
978:	learn: 0.0014335	test: 0.0697394	best: 0.0593624 (171)	total: 4m 6s	remaining: 5.3s
979:	learn: 0.0014335	test: 0.0697386	best: 0.0593624 (171)	total: 4m 7s	remaining: 5.04s
980:	learn: 0.0014335	test: 0.0697370	best: 0.0593624 (171)	total: 4m 7s	remaining: 4.79s
981:	learn: 0.0014335	test: 0.0697354	best: 0.0593624 (171)	total: 4m 7s	remaining: 4.54s
982:	learn: 0.0014335	test: 0.0697338	best: 0.0593624 (171)	total: 4m 7s	remaining: 4.29s
983:	learn: 0.0014335	test: 0.0697334	best: 0.0593624 (171)	total: 4m 8s	remaining: 4.04s
984:	learn: 0.0014335	test: 0.0697333	best: 0.0593624 (171)	total: 4m 8s	remaining: 3.78s
985:	learn: 

In [23]:
len(pred_arr)

6

In [24]:
df_sub = pd.DataFrame(columns=['ID','Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance'])
#'TITLE','ABSTRACT',
#df_sub['ID'] = df_test['ID']
#df_sub['TITLE'] = df_test['TITLE']
#df_sub['ABSTRACT'] = df_test['ABSTRACT']

for i,each in enumerate(df_sub.columns):
  if each!='ID':
    print (each)
    df_sub[each] = pred_arr[i-1]
df_sub['ID'] = df_test['ID']
#df_sub['TITLE'] = df_test['TITLE']
#df_sub['ABSTRACT'] = df_test['ABSTRACT']

Computer Science
Physics
Mathematics
Statistics
Quantitative Biology
Quantitative Finance


In [25]:
df_sub.to_csv('submission_prep_0.15_test.csv', index=False)

In [ ]:
df_sub.head()

In [26]:
f1_arr

[0.9894132914418415,
 0.9986220472440945,
 0.9400474871388999,
 0.935799096353893,
 0.8789531079607416,
 0.9931662870159453]